# Movimientos Table

This notebook contains an analysis made to the Movement table of the Historical Locality Archive of all the states of Mexico. It focuses on the column ORI_MODIF because this is the column that contains texts that explain the administrative modifications that the locality has gone through. The result of this notebook will be a table generated from the ORI_MODIF column that will serve to fill the continuity gaps present in the population table of the dataset (habitantes.csv).

## Inported Libraries and Configurations

In [1]:
import os
import pandas as pd

## Import data

In [2]:
# Define the types of columns
dtype_columnas = {
    'CLAVE' : 'int64',
    'CVE_GEOEST' : 'string',
    'INDICE' : 'int64',
    'NOM_LOC' : 'string',
    'NOM_MUN' : 'string',
    'CAT_POLI' : 'string',
    'CAT_ADMIVA'   : 'string',
    'ORI_MODIF' : 'string' 
}

In [3]:
# Import CSVs

movimientos = pd.read_csv('CSVs/movimientos.csv', dtype= dtype_columnas)

In [4]:
# General Info

#movimientos.dtypes                     # Types of columns
#movimientos.columns                    # Column names
#movimientos.info(show_counts=True)     # General info
#movimientos.head(3)

## Analisis de todas las columnas exepto ORI_MODIF

In [5]:
# Columna CLAVE

#movimientos.CLAVE.is_unique        # Contains unique values

In [6]:
# CVE_GEOEST Column

#movimientos.CVE_GEOEST.is_unique                       # Does not contain unique values
#max(movimientos.CVE_GEOEST.value_counts())             # Max frequency of a value is 24
#min(movimientos.CVE_GEOEST.value_counts())             # Min frequency of a value is 1
#movimientos.CVE_GEOEST.value_counts()                  # Value Frecuency - Most repeated value is 300130002
#movimientos.CVE_GEOEST.value_counts().value_counts()   # Frequency repetition count

In [10]:
# INDICE Column (It is NOT a table index)

#movimientos.INDICE.is_unique                       # Does not contain unique values
#max(movimientos.INDICE.value_counts())             # Max frecuency of a value is 409446
#min(movimientos.INDICE.value_counts())             # Min frequency of a value is 1
#movimientos.INDICE.value_counts()                  # Value Frecuency - Most repeated value is 1 followed by 2

In [8]:
# NOM_LOC Column

#movimientos.NOM_LOC.is_unique                       # Does not contain unique values
#max(movimientos.NOM_LOC.value_counts())             # Max frecuency of a value is 9721
#min(movimientos.NOM_LOC.value_counts())             # Min frequency of a value is 1
#movimientos.NOM_LOC.value_counts()                  # Value frecuency
#movimientos.NOM_LOC.value_counts().value_counts()   # Frequency repetition count

In [9]:
# NOM_MUN Column

#movimientos.NOM_MUN.is_unique                       # Does not contain unique values
#max(movimientos.NOM_MUN.value_counts())             # Max frecuency of a value is 20739
#min(movimientos.NOM_MUN.value_counts())             # Min frequency of a value is 1
#movimientos.NOM_MUN.value_counts()                  # Value Frecuency - Most repeated value is Ensenada followed by Mexicalli
#len(movimientos[movimientos.NOM_MUN.isnull()])      # There are 34 null values
#movimientos[movimientos.NOM_MUN.isnull()].head(3)   # There are no null values in CVE_GEOEST so the Municipality can be deduced
#movimientos[movimientos.NOM_MUN.isnull()]['ORI_MODIF'].value_counts()
                                                     # The value of ORI_MODIF when NOM_MUN is null

In [10]:
# CAT_POLI Column

#movimientos.CAT_POLI.is_unique                       # Does not contain unique values
#max(movimientos.CAT_POLI.value_counts())             # Max frecuency of a value is 1590564
#min(movimientos.CAT_POLI.value_counts())             # Min frequency of a value is 1
#movimientos.CAT_POLI.value_counts()                  # Frecuency values - Most repeated value is Indefinida
#movimientos[movimientos.CAT_POLI.isnull()]           # There is a null value

In [11]:
# CAT_ADMIVA Column

#movimientos.CAT_ADMIVA.is_unique                   # Does not contain unique values
#max(movimientos.CAT_ADMIVA.value_counts())         # Max frecuency of a value is 6800
#min(movimientos.CAT_ADMIVA.value_counts())         # Min frequency of a value is 2
#movimientos.CAT_ADMIVA.value_counts()              # Value Frecuency - Most repeated value is Delegación
#movimientos['CAT_ADMIVA'].isnull().sum()           # There are 2_419_988 null values
#movimientos['CAT_ADMIVA'].notnull().sum()          # 20541 are not null
#movimientos[movimientos.CAT_ADMIVA.isin([' '])].head(5)  # 106 have a space character as a value

## ORI_MODIF column analysis

### General overview

In [12]:
# Columna ORI_MODIF

#movimientos.ORI_MODIF.is_unique                 # Does not contain unique values (values repeat themselves)
#movimientos.ORI_MODIF.nunique()                 # Contains 68794 unique values
#movimientos.ORI_MODIF.isnull().value_counts()   # There are no null values
#movimientos.ORI_MODIF.value_counts()            # Value Frecuency - Most repeated value is "Censo de 2020."
                    

Data frame will be reducend only to column CLAVE and ORI_MODIF to make it less memory intensive.

In [13]:
# Reduce dataframe to CLAVE and ORI_MODIF columns
origen_modificacion = movimientos.loc[:,('CLAVE','ORI_MODIF')]

# Remove line jumps in ORI_MODIF
origen_modificacion.ORI_MODIF.replace('\r\n', ' ', regex=True, inplace=True)

# Trim the white spaces at the begining and end of the string
origen_modificacion.ORI_MODIF = origen_modificacion.ORI_MODIF.str.strip()


In [14]:
# Make a copy of original dataset to play with it
a_revisar = origen_modificacion.copy() 

In [15]:
#a_revisar.shape                                        # 2440529 rows and 2 columns 
#origen_modificacion.ORI_MODIF.value_counts().count()   # 63_087 unique values (original)
#a_revisar.ORI_MODIF.value_counts().count()             # 63_087 unique values (the copy)

### Spelling mistakes

There where typing mistakes in the process of digitalization. The detected words were added to individual lists of each specific word.

In [16]:
# Misspelling of the word censo
censo_ortografia = ['Ceso','Conso','Cenos','Cesno','Cense','Ceno',
                    'Censd','Cnso','Cneso','Censio','Cendo','Cemnso',
                    'Cemso', 'Enso', 'Ce nso']

# Cases to review of the word Censo
#a_revisar[a_revisar.ORI_MODIF.str.contains('Ce ')] # Se incluye
#a_revisar[a_revisar.ORI_MODIF.str.contains('Censó')].ORI_MODIF.to_list() # No se incluye

# # Misspelling of the word Conteo
conteo_ortografia = ['Conte','Centeo','COnteo','Comnteo']

# # Misspelling of the word Decreto
decreto_ortografia = ['Decreo','Decereto','Decreno','Decrero', 'decreto',
                      'Decrerto','Decrteo','Dedreto','Dercreto','Dto.']

# # Misspelling of the word Actualización
actualizacion_ortografia = ['Actuaización','Actualizacón', 'Actualizazión',
                            'Atualización', 'Actua ', 'Activación']

# Cases to review of the word Actualización
#a_revisar[a_revisar.ORI_MODIF.str.contains('Actua ')] # Se incluye
#a_revisar[a_revisar.ORI_MODIF.str.contains('Activación')] # Se incluye

# Case Bando only has one detected misspelling that can be included in the regular expresion
# [Bandpo]

# Misspelling of the word Oficio
oficio_ortografia = ['Ofiocio','Oficuio','Ofico']

# Misspelling of the word Aclaración
aclaracion_ortografia = ['Aclración','Aclaratoria']

# Cases to review of the word Aclaración
#a_revisar[a_revisar.ORI_MODIF.str.contains('Aclaratoria')] # It is included although it could be the word Declaratoria

### Origin clasification

In [17]:
# Prescense of key words in rows

# Total records in ORI_MODIF
#a_revisar.ORI_MODIF.count()                               # 2_440_529 matches in total                                                 

# Case Censo
#a_revisar[a_revisar.ORI_MODIF.str.contains('[Cc]enso')]   # 1_755_017 matches
#a_revisar[a_revisar.ORI_MODIF.str.contains('|'.join(censo_ortografia))].count() # 36 matches

# Case Conteo
#a_revisar[a_revisar.ORI_MODIF.str.contains('[Cc]onteo')]  # 555_005 matches 
#a_revisar[a_revisar.ORI_MODIF.str.contains('|'.join(conteo_ortografia))] # 555_ 012 matches

# Case Decreto
#a_revisar[a_revisar.ORI_MODIF.str.contains('[Dd]ecreto')]  # 55_235 matches
#a_revisar[a_revisar.ORI_MODIF.str.contains('|'.join(decreto_ortografia))].count() # 21 matches


# Case Actualización Geoestadística
#a_revisar[a_revisar.ORI_MODIF.str.contains('[Aa]ctualizaci[oó]n')]  # 40_522 matches
#a_revisar[a_revisar.ORI_MODIF.str.contains('|'.join(actualizacion_ortografia))].count() # 18 matches

# Case Bando
#a_revisar[a_revisar.ORI_MODIF.str.contains('[Bb]and[op]')]  # 16_777 matches

# Case Ley
#a_revisar[a_revisar.ORI_MODIF.str.contains('[Ll]ey')]  #  9_478 matches

# Case Oficio
#a_revisar[a_revisar.ORI_MODIF.str.contains('[Oo]ficio')]  # 5855 matches
#a_revisar[a_revisar.ORI_MODIF.str.contains('|'.join(oficio_ortografia))].count() # 3 matches

# Case Periódico
#a_revisar[a_revisar.ORI_MODIF.str.contains('[Pp]eri[oó]dico')]  # 7_776 matches

# Case Aclaración
#a_revisar[a_revisar.ORI_MODIF.str.contains('[Aa]claraci[oó]n')]  # 400 matches
#a_revisar[a_revisar.ORI_MODIF.str.contains('|'.join(aclaracion_ortografia))].count() # 2 matches


# Case Constitución... Articulo 129 se junta aqui.. revisar
#a_revisar[a_revisar.ORI_MODIF.str.contains('(c|C)(onstitución)')] # 602 matches (Case that shares records with actualizacion)

# Case Reformas
#a_revisar[a_revisar.ORI_MODIF.str.contains('[Rr]eformas')]  # 389 matches


# Case Presidencial
#a_revisar[a_revisar.ORI_MODIF.str.contains('(P|p)(residencial)')]  # 306 matches

# Integración
#a_revisar[a_revisar.ORI_MODIF.str.contains('((l|L)ista|(i|I)ntegración)')]# 402 matches (Case that shares records with actualziacion)

# Case Sentencias
#a_revisar[a_revisar.ORI_MODIF.str.contains('[Ss]entencia')].count() # 17 matches

# Case Acuerdos
#a_revisar[a_revisar.ORI_MODIF.str.contains('[aA]cuerdo')].count()  # 35 matches

# Case Documento
#a_revisar[a_revisar.ORI_MODIF.str.contains('[dD]ocumento')].count()  # 36 matches

# Case Reglamento
#a_revisar[a_revisar.ORI_MODIF.str.contains('[rR]eglamento')].count() # 93 matches


# Case Equivalencia
#a_revisar[a_revisar.ORI_MODIF.str.contains('[eE]quivalencia')].count() # 36 matches

#Case Intercensal
#a_revisar[a_revisar.ORI_MODIF.str.contains('[iI]ntercensal')].count() # 45 matches

# Case Resolución
#a_revisar[a_revisar.ORI_MODIF.str.contains('[rR]esoluci[oó]n')] # 259 matches (Has many cases in common with resolucion presidencial)

# Case Inventario
#a_revisar[a_revisar.ORI_MODIF.str.contains('[iI]nventario')].count() # 8 matches


# Case Sindicatura
#a_revisar[a_revisar.ORI_MODIF.str.contains('(s|S)(indicatura)')] # 1 matches

# Case Cambio
#a_revisar[a_revisar.ORI_MODIF.str.contains('(C|c)(ambio)')] # 235947 matches (Intersects with many inportant ones including censo, decreto y actualizacion  )


#a_revisar[a_revisar.ORI_MODIF.str.contains('[Oo]axaca')] # 8020 matches (Intersects with conteo)


A grandes rasgos, las siguientes palabras clave se pueden utilizar para dividir los registros en diferentes tipos de origen. Algunas palabras se pueden generalizar en un solo tipo y existen traslapes que se tienen que analizar con mas cuidado. Las divisiones acontinuacion se utilizarán para dividir los registros y hacer análisis mas finos.

In [18]:
# Total de registros: 2440529, Total of unique values: 63087

# Case Censo
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[Cc]enso')]    # Remaining matches: 685512 
                                                                        # Remaining unique values: 40528
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('|'.join(censo_ortografia))]    # Remaining matches: 685483
                                                                                        # Remaining unique values: 40502

# Case Conteo
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[Cc]onteo')]   # Remaining matches: 130486
                                                                        # Remaining unique values: 31775
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('|'.join(conteo_ortografia))]   # Remaining matches: 130477
                                                                                        # Remaining unique values: 31769

# Case Decreto
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[Dd]ecreto')]      # Remaining matches: 76815
                                                                            # Remaining unique values: 14712
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('|'.join(decreto_ortografia))]      # Remaining matches: 76805
                                                                                            # Remaining unique values: 14702

# Case Actualizacion Geoestadística
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[Aa]ctualizaci[oó]n')]     # Remaining matches: 36324
                                                                                    # Remaining unique values: 7619
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('|'.join(actualizacion_ortografia))]    # Remaining matches: 36306
                                                                                                # Remaining unique values: 7606

# Case Bando
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[Bb]and[op]')]     # Remaining matches: 19533
                                                                            # Remaining unique values: 4631

# Case Ley
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[Ll]ey')]      # Remaining matches: 10670
                                                                        # Remaining unique values: 3214

# # Case oficio
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[Oo]ficio')]   # Remaining matches: 5434
                                                                        # Remaining unique values: 1323
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('|'.join(oficio_ortografia))]   # Remaining matches: 5431
                                                                                        # Remaining unique values: 1320

# Case Periódico
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[Pp]eri[oó]dico')]     # Remaining matches: 4776
                                                                                # Remaining unique values: 941

# Case Aclaración
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[Aa]claraci[oó]n')]    # Remaining matches: 4402
                                                                                # Remaining unique values: 908
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('|'.join(aclaracion_ortografia))]   # Remaining matches: 4400
                                                                                            # Remaining unique values: 906

# Case Reformas
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[Rr]eformas')]     # Remaining matches: 4132
                                                                            # Remaining unique values: 899

# Case Inventario 
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[iI]nventario')]   # Remaining matches: 4124
                                                                            # Remaining unique values: 896

# Case Sentencia
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[Ss]entencia')]    # Remaining matches: 4107
                                                                            # Remaining unique values: 879

# Case Reglamento
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[rR]eglamento')]   # Remaining matches: 4050
                                                                            # Remaining unique values: 861

# Case Intercensal
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[iI]ntercensal')]      # Remaining matches: 4012
                                                                                # Remaining unique values: 845

# Case Documento
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[dD]ocumento')]    # Remaining matches: 3977
                                                                            # Remaining unique values: 821

# Case Acuerdo
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[aA]cuerdo')]      # Remaining matches: 3946
                                                                            # Remaining unique values: 795

# Case Equivalencia
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[eE]quivalencia')]     # Remaining matches: 3910
                                                                                # Remaining unique values: 782

# Case Integración
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[lL]ista|[iI]ntegraci[oó]n')]      # Remaining matches: 3840
                                                                                            # Remaining unique values: 750

# Case Resolución ... se podria juntar con sentencia?
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[rR]esoluci[oó]n')]    # Remaining matches: 3533
                                                                                # Remaining unique values: 561

# Case Estado de Oaxaca
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[oO]axaca')]   # Remaining matches: 877
                                                                        # Remaining unique values: 426

# Case Constitución
a_revisar = a_revisar[~a_revisar.ORI_MODIF.str.contains('[Cc]onstituci[oó]n')]      # Remaining matches: 491
                                                                                    # Remaining unique values: 147



In [38]:
# Remaining records and unique value counts
registros_restantes = a_revisar.ORI_MODIF.count()
cantidad_valores_unicos = a_revisar.ORI_MODIF.value_counts().count()

print(f'Remaining matches: {registros_restantes}, Remaining unique values: {cantidad_valores_unicos}')

Registros restantes: 491, Cantidad de valores unicos: 147


The techniques in which key word detection was based on were the extraction of the records first word, first letter and specific words. The remaining unique 147 values that didn’t fit any of the selected keywords will be reviewed in a later stage with other individual records that will be accumulating in further analysis.

In [24]:
# First word list in remaining unique values

# Extract the first word in the strings of ORI_MODIF
primera_palabra = a_revisar.ORI_MODIF.str.extract(r'^([^\s]+)').value_counts()

# Resetear el indice de una tupla a solo la primera palabra
# Reset the index form a tuple to only the string
primera_palabra.index = primera_palabra.index.get_level_values(0)

# Remaining first words
#primera_palabra

In [ ]:
# Remaining unique values
valores_unicos_restantes = a_revisar.ORI_MODIF.value_counts().index

In [ ]:
# Explore the remaining unique values
#valores_unicos_restantes[0:20]

In [34]:
# Divide the remaining values by letter
primera_letra = 'C'

# Unique values that start with this specific letter
#a_revisar[a_revisar.ORI_MODIF.str.startswith(primera_letra)]

# First string words that start with this specific letter
#primera_palabra[primera_palabra.index.str.startswith(primera_letra)]

## The diferent origin types

The diferent origin types that have been detected are as follows: 
* Census Events
* Decrees
* (...)



### Census Events

The first orgin event to be analyzed is the Census Event. The vast majority of modifications have this origin. Of the total records of 2_440_529, only 130_477 remain after extracting those containing the words Census and Count. The two great types of census event are these two categories.

Within the Census category, the population type is the most common one, but there are also cases where the modification comes from an economic or from a buildings census. All events contain a temporary dimension, they were carried out in aspecific year. The first population census was carried out in the year 1900 and the most recent one is from the year 2020.

In [ ]:
# Census

In [13]:
# Count

## To be organized...

In [77]:


empieza_con_censo = origen_modificacion.loc[origen_modificacion.ORI_MODIF.str.match('^Censo')].ORI_MODIF


In [127]:
#empieza_con_censo.count()   #Hay diferencia de 14 con las que empiezan con censo seguidas de un espacio
#empieza_con_censo[~empieza_con_censo.str.match('^Censo de \d{4}')].count() # 3920 Registros que no siguen Censo de y 4 digitos
#empieza_con_censo[~empieza_con_censo.str.match('^Censo de \d{4}')].value_counts().count() # 330 valores unicos
#empieza_con_censo[~empieza_con_censo.str.match('^Censo de \d{4}')].value_counts()[:20]   # censo e edificios es el mas comun 
empieza_con_censo[(~empieza_con_censo.str.match('^Censo de \d{4}')) & 
                  (~empieza_con_censo.str.match('.+(edificios|Edificios|Económico)'))].value_counts()[160:180]



Censo de1930. Cambio de nombre del municipio. ...    1
Censo de1930. Censó con la localidad Paso Plat...    1
Censo de1930. Censó con la localidad Manuel Ma...    1
Censo de1910. Cambio de categoría política.          1
Censo 2010. Viene del municipio Villa de Cos (...    1
Name: ORI_MODIF, dtype: Int64

In [70]:
origen_modificacion.loc[origen_modificacion.ORI_MODIF.str.match('^Censo de \d{4}')].value_counts()

CLAVE    ORI_MODIF                                          ORIGEN
4861341  Censo de 1960.                                     Censo     1
6481651  Censo de 1940. Cambio de nombre de localidad. ...  Censo     1
6481649  Censo de 1921.                                     Censo     1
6481648  Censo de 1910.                                     Censo     1
6481647  Censo de 1900.                                     Censo     1
                                                                     ..
5692074  Censo de 1990.                                     Censo     1
5692072  Censo de 2000.                                     Censo     1
5692070  Censo de 2020.                                     Censo     1
5692069  Censo de 2010.                                     Censo     1
7301869  Censo de 2010.                                     Censo     1
Length: 1749214, dtype: int64

In [25]:
origen_modificacion.ORI_MODIF.str.extract(r'^[^\d]*(\d{4})').value_counts()

2010    283965
1995    280263
2000    278869
2020    275821
2005    274825
         ...  
1788         1
1787         1
1785         1
1781         1
9598         1
Length: 671, dtype: int64

In [116]:
len(todo[todo.str.match(r'.+ctualizac.+')].value_counts())

9021

In [34]:
# Solo los valores que actualizaron la cantidad de población por un evento censal
actualizacion_censal_pob = todo[todo.str.match(r'^(Censo de |Conteo de )\d{4}\.$')]

In [35]:
# Valores que no son actualizaciones censales de poblacion
otro = todo[~todo.str.match(r'^(Censo de |Conteo de )\d{4}\.$')]

In [36]:
actualizacion_censal_tipo = todo[todo.str.match(r'^(Censo de |Conteo de )\d{4}\..+')]

In [72]:
actualizacion_tipo = otro[~todo.str.match(r'^(Censo de |Conteo de )\d{4}\..+')]

In [84]:
(actualizacion_tipo.value_counts() < 2).value_counts()

True     28164
False     7560
Name: ORI_MODIF, dtype: Int64

In [35]:
actualizacion_censal_tipo.replace(r'^(Censo de |Conteo de )\d{4}\.?\s+', '',regex=True).value_counts()
#'^(Censo de |Conteo de )\d{4}\.', ''

NameError: name 'actualizacion_censal_tipo' is not defined

In [32]:
# Comprobaciones
#actualizacion_censal_tipo.str.startswith(('Censo', 'Conteo')).value_counts()

True    870288
Name: ORI_MODIF, dtype: Int64

In [22]:
# actualizacion_censal.value_counts()     # Sumandolos son los valores mas comunes de origen de modificacion.

Censo de 2020.     188742
Censo de 2000.     178687
Censo de 2010.     177892
Conteo de 2005.    171106
Conteo de 1995.    123676
Censo de 1990.     108950
Censo de 1980.      98520
Censo de 1970.      76107
Censo de 1960.      67008
Censo de 1950.      60354
Censo de 1940.      54146
Censo de 1930.      36823
Censo de 1900.      32516
Censo de 1910.      30783
Censo de 1921.      26596
Censo de 1995.         27
Censo de 1939.         22
Censo de 2005.          9
Conteo de 1990.         6
Conteo de 1930.         2
Conteo de 2000.         1
Conteo de 1925.         1
Conteo de 1940.         1
Conteo de 1921.         1
Name: ORI_MODIF, dtype: Int64

In [43]:

todo = movimientos[movimientos.ORI_MODIF.str.match(r'^(Censo de |Conteo de )\d{4}\.$')].ORI_MODIF.str.replace(r'\r\n', ' ')
actualizacion_censal = movimientos[movimientos.ORI_MODIF.str.match(r'^(Censo de |Conteo de )\d{4}\..+')].ORI_MODIF
#otros = movimientos[~movimientos.ORI_MODIF.str.match(r'^(Censo de |Conteo de )\d{4}\.$')].ORI_MODIF



C:\Users\Lorrain\AppData\Local\Temp\ipykernel_5516\2214415388.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  todo = movimientos[movimientos.ORI_MODIF.str.match(r'^(Censo de |Conteo de )\d{4}\.$')].ORI_MODIF.str.replace(r'\r\n', ' ')


In [44]:
actualizacion_censal.value_counts()

Censo de 2020.
Localidad deshabitada.               86232
Conteo de 2005.
Localidad deshabitada.              72883
Censo de 2010.
Localidad deshabitada.               71913
Censo de 2000.
Localidad deshabitada.               50078
Conteo de 1995.
Localidad deshabitada.              44475
                                                     ...  
Censo de 1950.

Localidad  deshabitada.                1
Conteo de 1995.
Se fusiona a  Sabidos (0077)....        1
Conteo de 1995.
Cambio de nombre de localidad...        1
Censo de 1980.
Pasa al municipio El Higo (205...        1
Censo de 2010.
Viene del municipio Villa de C...        1
Name: ORI_MODIF, Length: 32338, dtype: Int64

In [40]:
list(censal_mas_info.str.replace(r'\r\n', ' ').unique())

C:\Users\Lorrain\AppData\Local\Temp\ipykernel_5516\344913569.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  list(censal_mas_info.str.replace(r'\r\n', ' ').unique())


['Censo de 2020. Localidad deshabitada.',
 'Censo de 2010. Localidad deshabitada.',
 'Censo de 2010. Cambio de nombre de localidad.',
 'Conteo de 2005. Localidad deshabitada.',
 'Conteo de 2005. Localidad rehabilitada.',
 'Conteo de 2005. Localidad fusionada a Baños la Cantera (0114).',
 'Conteo de 2005. Cambio de nombre de localidad.',
 'Conteo de 1995. Baja de localidad.',
 'Censo de 2000. Localidad deshabitada.',
 'Censo de 2010. Baja de localidad.',
 'Censo de 2000. Cambio de nombre de localidad.',
 'Censo de 2000. Baja de localidad.',
 'Censo de 2000. Localidad rehabilitada.',
 'Conteo de 1995. Cambio de nombre de localidad. Localidad con dos viviendas.',
 'Censo de 2010. Localidad rehabilitada.',
 'Conteo de 1995. Baja de localidad. Pertenece al Estado de Jalisco 14.',
 'Conteo de 1995. Cambio de nombre de localidad.',
 'Conteo de 1995. Localidad con una vivienda.',
 'Conteo de 1995. Localidad de dos viviendas.',
 'Conteo de 1995. Localidad con dos viviendas.',
 'Censo de 2005. L

In [38]:
list(censal_mas_info.str.replace(r'^(Censo de |Conteo de )\d{4}\.', '').unique()[20:])

C:\Users\Lorrain\AppData\Local\Temp\ipykernel_5516\3409716397.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  list(censal_mas_info.str.replace(r'^(Censo de |Conteo de )\d{4}\.', '').unique()[20:])


['\r\nCambio de nombre de localidad.\r\nLocalidad con una vivienda.',
 '\r\nLocalidad rehabilitada.\r\nLocalidad con una vivienda.',
 '\r\nLocalidad de una vivienda.',
 '\r\nCambio de nombre de localidad.\r\nLocalidad de dos viviendas.',
 '\r\nSe conurba a Ojocaliente 0055.',
 '\r\nHasta este evento estuvo registrada con clave (010010383).',
 '\r\nA partir de este evento cambia a la clave 010090051.',
 '\r\nCambio de nombre de localidad.\r\nCambio de categoría política.',
 'Se conurba a Ojocaliente (0055).',
 '\r\nLocalidad de dos Viviendas.',
 '\r\nLocalidad rehabilitada.\r\nLocalidad con dos viviendas.',
 '\r\nLocalidad rehabilitada.\r\nCambio de categoría política.',
 '\r\nHasta este evento estuvo registrada con clave (010010317).',
 '\r\nSe conurba a Pabellón de Arteaga 0001.',
 ' ',
 '\r\nHasta este evento estuvo registrada con clave 010050094.',
 '\r\nA partir de este evento cambia a la clave (010011112).\r\nLocalidad con dos viviendas.',
 '\r\nCambio de categoría política.',
 '\

In [19]:
#puramente_censal.value_counts()
otros.value_counts()

Censo de 2020.
Localidad deshabitada.               86232
Conteo de 2005.
Localidad deshabitada.              72883
Censo de 2010.
Localidad deshabitada.               71913
Censo de 2000.
Localidad deshabitada.               50078
Conteo de 1995.
Localidad deshabitada.              44475
                                                     ...  
Censo de 1970.
Hasta este evento estuvo regis...        1
Conteo de 1995.
Viene del municipio Ciudad Va...        1
Censo de 1990.
Se censó con la localidad La C...        1
Decreto No. 225 del 2 de diciembre de 1994. 
...        1
Censo de 2010.
Viene del municipio Villa de C...        1
Name: ORI_MODIF, Length: 68770, dtype: Int64